# manuela CW measurements

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import plotHelperLatex
plotHelperLatex.setMatplotSettings()

allSpectra = np.genfromtxt(r"C:\Users\M\Documents\Books\masterprojectinformation\images\2021-07-05ManuelaSQIB.txt", skip_header=2, usecols=(0,4))

wavelengths = allSpectra[:,0]
boolArr = wavelengths < 1000
wavelengths = wavelengths[boolArr]
relTransmission = allSpectra[boolArr,1]

fig, ax = plt.subplots(1,1, figsize=plotHelperLatex.figSizer(1,3), dpi = 288)
ax.plot(wavelengths, relTransmission*100)
ax.set_yscale('log')
ax.set_ylabel('transmission / %')
ax.set_xlabel('wavelength / nm')
plt.show()

### Red LEDs

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import plotHelperLatex
plotHelperLatex.setMatplotSettings()

allSpectra = np.genfromtxt(r"C:\Users\M\Documents\Books\masterprojectinformation\images\SpectrumRedLEDsforCalibration.txt", skip_header=17, skip_footer=1)

wavelengths = allSpectra[:,0]
boolArr = (wavelengths > 500) & (wavelengths < 800)
emission = allSpectra[:,1]

fig, ax = plt.subplots(1,1, figsize=plotHelperLatex.figSizer(1,3), dpi = 288)
ax.plot(wavelengths[boolArr], emission[boolArr]/np.max(emission))
ax.set_ylabel('emission / a.u.')
ax.set_xlabel('wavelength / nm')
plt.show()